In [1]:
import numpy as np

In [2]:
np.set_printoptions(precision=4)

In [3]:
m = np.array([2, 1, 2, 2])
mu = np.array([10, 3, 7, 7])
M = 4
N = 4

In [4]:
alpha = np.array([4/3, 56/297, 119/297, 56/297])
alpha

array([1.3333, 0.1886, 0.4007, 0.1886])

In [5]:
omega = alpha / alpha.sum()
omega, omega.sum()

(array([0.6316, 0.0893, 0.1898, 0.0893]), 0.9999999999999999)

In [6]:
states = np.array([[4,0,0,0], [3,1,0,0], [3,0,1,0], [3,0,0,1], [2,2,0,0], [2,1,1,0], [2,1,0,1], [2,0,2,0], [2,0,1,1], [2,0,0,2], 
                   [1,3,0,0], [1,2,1,0], [1,2,0,1], [1,1,2,0], [1,1,1,1], [1,1,0,2], [1,0,3,0], [1,0,2,1], [1,0,1,2], [1,0,0,3], 
                   [0,4,0,0], [0,3,1,0], [0,3,0,1], [0,2,2,0], [0,2,1,1], [0,2,0,2], [0,1,3,0], [0,1,2,1], [0,1,1,2], [0,1,0,3], 
                   [0,0,4,0], [0,0,3,1], [0,0,2,2], [0,0,1,3], [0,0,0,4]])

In [7]:
def calc_mu(j, k):
    return k * mu[j - 1] if k < m[j - 1] else m[j - 1] * mu[j - 1]

def calc_z(i, n):
    z = omega[i - 1] ** n
    for j in range(1, n + 1):
        z /= calc_mu(i, j)
    return z

def calc_g(states):
    g = 0
    for state in states:
        mult = 1
        for i in range(1, M + 1):
            mult *= calc_z(i, state[i - 1])
        g += mult
    return g

def calc_p(state, G):
    p = 1
    for i in range(1, M + 1):
        p *= calc_z(i, state[i - 1])
    return p / G

In [8]:
G = calc_g(states)
p = np.zeros(states.shape[0])
for i in range(states.shape[0]):
    p[i] = calc_p(states[i], G)
p, p.sum()

(array([0.0931, 0.0878, 0.0799, 0.0376, 0.0828, 0.0754, 0.0355, 0.0343,
        0.0323, 0.0076, 0.078 , 0.071 , 0.0334, 0.0324, 0.0304, 0.0072,
        0.0147, 0.0139, 0.0065, 0.0015, 0.0368, 0.0335, 0.0158, 0.0153,
        0.0144, 0.0034, 0.0069, 0.0065, 0.0031, 0.0007, 0.0032, 0.003 ,
        0.0014, 0.0007, 0.0002]), 0.9999999999999999)

## Расчет узлов замкнутой сети

In [9]:
n = np.zeros(M) # n
nw = np.zeros(M) # n_ож
ns = np.zeros(M) # n_обсл
for j in range(M):
    nj = states[:,j]
    n[j] = np.multiply(nj, p).sum()
    nw[j] = np.multiply(np.where(nj - m[j] > 0, nj - m[j], 0), p).sum()
    ns[j] = np.multiply(np.where(nj > m[j], m[j], nj), p).sum()
n, nw, ns, n - (nw + ns)

(array([1.8131, 1.2552, 0.6413, 0.2904]),
 array([0.3915, 0.5851, 0.031 , 0.0032]),
 array([1.4216, 0.6701, 0.6103, 0.2872]),
 array([0., 0., 0., 0.]))

In [10]:
t = n / ns / mu
tw = nw / ns / mu
ts = t - tw
t, tw, ts, t - (tw + ts)

(array([0.1275, 0.6244, 0.1501, 0.1445]),
 array([0.0275, 0.291 , 0.0073, 0.0016]),
 array([0.1   , 0.3333, 0.1429, 0.1429]),
 array([0., 0., 0., 0.]))